In [1]:
## where to write temporary files 
TMP_PREFIX = "tmp/"

## csv file to find the annotations
ANNOTATION_FILE = "toy_data/test_annotation.csv"



mains steps: 
1. check overlap between parts
2. check overlap between cutout and parts extremities: push part extremities 
3. from cut out to sub parts
4. generate split and merge commands
5. apply commands






In [2]:
from collections import defaultdict
import os 
import pandas as pd
import numpy as np

from utils import *

In [3]:
df_annot = pd.read_csv(ANNOTATION_FILE)
df_annot['start_second'] = df_annot.start.apply(time_to_seconds) ## convert time to seconds
df_annot['stop_second'] = df_annot.stop.apply(time_to_seconds)   ## convert time to seconds


## check that all elements start before they stop:
PB = df_annot.loc[ df_annot.stop_second < df_annot.start_second , :]
if PB.shape[0]>0:
    print("PROBLEM: some elements stop before they start! please fix this")
    print( PB )
    
df_annot

,source,start,stop,destination,start_second,stop_second
0,vid1,00:00:00,01:30:45,part1,0,5445
1,vid1,01:31:00,02:02:00,part2,5460,7320
2,vid2,00:02:12,01:00:00,part2,132,3600
3,vid1,00:10:00,00:25:30,OUT,600,1530
4,vid1,01:29:00,01:32:00,OUT,5340,5520
5,vid2,00:00:00,00:01:00,OUT,0,60
6,vid2,01:01:00,01:12:00,part3_overlap,3660,4320
7,vid2,01:10:00,01:15:00,part3_overlap,4200,4500
8,vid1,01:28:00,01:30:00,OUT,5280,5400


In [4]:
## separate parts and cut out
df_annot_parts = df_annot.loc[ df_annot.destination != "OUT" , :].copy().reset_index(drop=True)
df_annot_OUT = df_annot.loc[ df_annot.destination == "OUT" , :].copy().reset_index(drop=True)


In [5]:
## check if some parts overlap :
check_overlap( df_annot_parts )


  source     start      stop    destination  start_second  stop_second
3   vid2  01:01:00  01:12:00  part3_overlap          3660         4320
4   vid2  01:10:00  01:15:00  part3_overlap          4200         4500
***


In [6]:
## merge overlapping cutouts 
df_annot_OUT = merge_overlapping_elements(df_annot_OUT)

##sort cutout by video and start!!
df_annot_OUT.sort_values( by = ['source' , 'start_second']  , inplace=True)


merging cutout rows: 3 and 1


In [7]:
# applying cutouts :
#  * overlaps is across start or stop of part -> move part start or stop
#  * otherwise part is split in 2
df_subparts = associate_parts_and_cutout(df_annot_parts,df_annot_OUT)
df_subparts

,source,start,stop,destination,start_second,stop_second
0,vid1,NaN,NaN,part1,0,600
1,vid1,NaN,NaN,part1,1530,5280
2,vid1,NaN,NaN,part2,5520,7320
3,vid2,NaN,NaN,part2,132,3600
4,vid2,NaN,NaN,part3_overlap,3660,4320
5,vid2,NaN,NaN,part3_overlap,4200,4500


In [8]:
cmds = create_commands( df_subparts , prefix = TMP_PREFIX )
cmds

{'part1': ['ffmpeg -ss 00:00:00 -to 00:10:00 -i vid1 tmp/destination0_clip0.mp4',
  'ffmpeg -ss 00:25:30 -to 01:28:00 -i vid1 tmp/destination0_clip1.mp4',
  'ffmpeg -f concat -safe 0 -i destination0.clipFiles.txt -c copy part1.mp4',
  'rm tmp/destination0_clip0.mp4',
  'rm tmp/destination0_clip1.mp4'],
 'part2': ['ffmpeg -ss 01:32:00 -to 02:02:00 -i vid1 tmp/destination1_clip0.mp4',
  'ffmpeg -ss 00:02:12 -to 01:00:00 -i vid2 tmp/destination1_clip1.mp4',
  'ffmpeg -f concat -safe 0 -i destination1.clipFiles.txt -c copy part2.mp4',
  'rm tmp/destination1_clip0.mp4',
  'rm tmp/destination1_clip1.mp4'],
 'part3_overlap': ['ffmpeg -ss 01:01:00 -to 01:12:00 -i vid2 tmp/destination2_clip0.mp4',
  'ffmpeg -ss 01:10:00 -to 01:15:00 -i vid2 tmp/destination2_clip1.mp4',
  'ffmpeg -f concat -safe 0 -i destination2.clipFiles.txt -c copy part3_overlap.mp4',
  'rm tmp/destination2_clip0.mp4',
  'rm tmp/destination2_clip1.mp4']}

In [9]:
# apply commands - this may take a long while 
for part , commands in cmds.items():
    print('creating',part)
    for c in commands:
        os.system(c)

creating part1


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

creating part2


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

creating part3_overlap


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e